In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import gc

%matplotlib inline

In [2]:
import statsmodels

In [3]:
df = pd.read_csv('FAO.csv', encoding = 'mbcs' ) 

In [4]:
df.head(100)

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,...,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,...,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200
5,AFG,2,Afghanistan,2514,Maize and products,5142,Food,1000 tonnes,33.94,67.71,...,231.0,67.0,82.0,67.0,69.0,71.0,82.0,73.0,77,76
6,AFG,2,Afghanistan,2517,Millet and products,5142,Food,1000 tonnes,33.94,67.71,...,15.0,21.0,11.0,19.0,21.0,18.0,14.0,14.0,14,12
7,AFG,2,Afghanistan,2520,"Cereals, Other",5142,Food,1000 tonnes,33.94,67.71,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
8,AFG,2,Afghanistan,2531,Potatoes and products,5142,Food,1000 tonnes,33.94,67.71,...,276.0,294.0,294.0,260.0,242.0,250.0,192.0,169.0,196,230
9,AFG,2,Afghanistan,2536,Sugar cane,5521,Feed,1000 tonnes,33.94,67.71,...,50.0,29.0,61.0,65.0,54.0,114.0,83.0,83.0,69,81


In [5]:
df1 = df.loc[:, ~(df.columns).isin(['Area Abbreviation','Area Code', 'Item Code', 'Element Code', 'Unit'])]

In [6]:
Aggr_sum = df1.groupby(['Area', 'Item', 'Element']).agg(sum).reset_index()
print(Aggr_sum)

              Area                             Item Element  latitude  \
0      Afghanistan              Alcoholic Beverages    Food     33.94   
1      Afghanistan                      Animal fats    Food     33.94   
2      Afghanistan              Apples and products    Food     33.94   
3      Afghanistan                          Bananas    Food     33.94   
4      Afghanistan              Barley and products    Feed     33.94   
5      Afghanistan              Barley and products    Food     33.94   
6      Afghanistan                             Beer    Food     33.94   
7      Afghanistan             Beverages, Alcoholic    Food     33.94   
8      Afghanistan                      Bovine Meat    Food     33.94   
9      Afghanistan                     Butter, Ghee    Food     33.94   
10     Afghanistan         Cereals - Excluding Beer    Feed     33.94   
11     Afghanistan         Cereals - Excluding Beer    Food     33.94   
12     Afghanistan                   Cereals, Other

In [7]:
Aggr_sum=Aggr_sum.fillna(0)

In [8]:
Aggr_sum_no_att = Aggr_sum.loc[:, ~(Aggr_sum.columns).isin(['latitude', 'longitude'])]

In [9]:
Aggr_sum_no_att.shape

(21018, 56)

In [10]:
no_att_by_country = Aggr_sum_no_att.groupby(['Area', 'Element']).agg(sum).reset_index()
print(no_att_by_country)

                                   Area Element     Y1961     Y1962     Y1963  \
0                           Afghanistan    Feed     720.0     720.0     736.0   
1                           Afghanistan    Food    8761.0    8694.0    8458.0   
2                               Albania    Feed      94.0     108.0     124.0   
3                               Albania    Food    1612.0    1641.0    1643.0   
4                               Algeria    Feed      83.0      94.0      63.0   
5                               Algeria    Food    7405.0    7141.0    6798.0   
6                                Angola    Feed     118.0     118.0     116.0   
7                                Angola    Food    4716.0    4657.0    5124.0   
8                   Antigua and Barbuda    Feed       2.0       2.0       2.0   
9                   Antigua and Barbuda    Food      90.0      92.0     103.0   
10                            Argentina    Feed    9552.0    7553.0    6527.0   
11                          

In [11]:
by_country = no_att_by_country.groupby(['Area']).agg(sum).reset_index()

In [12]:
df_all = Aggr_sum_no_att

In [13]:
data = by_country

In [14]:
data['total'] = (data['Y1961'] + data['Y1962'] + data['Y1963'] + data['Y1964'] + data['Y1965'] + data['Y1966'] + 
    data['Y1967'] + data['Y1968'] + data['Y1969'] + data['Y1970'] + data['Y1971'] + data['Y1972'] + data['Y1973'] +
    data['Y1974'] + data['Y1975'] + data['Y1976'] + data['Y1977'] + data['Y1978'] + data['Y1979'] + data['Y1980'] + 
    data['Y1981'] + data['Y1982'] + data['Y1983'] + data['Y1984'] + data['Y1985'] + data['Y1986'] + data['Y1987'] + 
    data['Y1988'] + data['Y1989'] + data['Y1990'] + data['Y1991'] + data['Y1992'] + data['Y1993'] + data['Y1994'] + 
    data['Y1995'] + data['Y1996'] + data['Y1997'] + data['Y1998'] + data['Y1999'] + data['Y2000'] + 
    data['Y2001'] + data['Y2002'] + data['Y2003'] + data['Y2004'] + data['Y2005'] + data['Y2006'] + data['Y2007'] + 
    data['Y2008'] + data['Y2009'] + data['Y2010'] + data['Y2011'] + data['Y2012'] + data['Y2013'] )

In [15]:
data.head()

,Area,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,...,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,total
0,Afghanistan,9481.0,9414.0,9194.0,10170.0,10473.0,10169.0,11289.0,11508.0,11815.0,...,17658.0,18317.0,19248.0,19381.0,20661.0,21030.0,21100.0,22706,23007,689162.0
1,Albania,1706.0,1749.0,1767.0,1889.0,1884.0,1995.0,2046.0,2169.0,2230.0,...,6719.0,6911.0,6744.0,7168.0,7316.0,7907.0,8114.0,8221,8271,237202.0
2,Algeria,7488.0,7235.0,6861.0,7255.0,7509.0,7536.0,7986.0,8839.0,9003.0,...,49562.0,51067.0,49933.0,50916.0,57505.0,60071.0,65852.0,69365,72161,1530613.0
3,Angola,4834.0,4775.0,5240.0,5286.0,5527.0,5677.0,5833.0,5685.0,6219.0,...,26696.0,28247.0,29877.0,32053.0,36985.0,38400.0,40573.0,38064,48639,706016.0
4,Antigua and Barbuda,92.0,94.0,105.0,95.0,84.0,73.0,64.0,59.0,68.0,...,115.0,110.0,122.0,115.0,114.0,115.0,118.0,113,119,4446.0


In [16]:
now = no_att_by_country

In [17]:
now

,Area,Element,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,Afghanistan,Feed,720.0,720.0,736.0,740.0,720.0,724.0,788.0,826.0,...,704.0,1184.0,1342.0,1392.0,1294.0,1616.0,1388.0,1192.0,1522,1536
1,Afghanistan,Food,8761.0,8694.0,8458.0,9430.0,9753.0,9445.0,10501.0,10682.0,...,15838.0,16474.0,16975.0,17856.0,18087.0,19045.0,19642.0,19908.0,21184,21471
2,Albania,Feed,94.0,108.0,124.0,122.0,95.0,197.0,202.0,229.0,...,990.0,994.0,1047.0,959.0,1075.0,1134.0,1334.0,1334.0,1312,1319
3,Albania,Food,1612.0,1641.0,1643.0,1767.0,1789.0,1798.0,1844.0,1940.0,...,5647.0,5725.0,5864.0,5785.0,6093.0,6182.0,6573.0,6780.0,6909,6952
4,Algeria,Feed,83.0,94.0,63.0,98.0,84.0,55.0,74.0,130.0,...,4575.0,4401.0,4599.0,4252.0,3436.0,4839.0,5804.0,7477.0,8549,8706
5,Algeria,Food,7405.0,7141.0,6798.0,7157.0,7425.0,7481.0,7912.0,8709.0,...,44044.0,45161.0,46468.0,45681.0,47480.0,52666.0,54267.0,58375.0,60816,63455
6,Angola,Feed,118.0,118.0,116.0,132.0,128.0,128.0,148.0,148.0,...,9622.0,9814.0,10004.0,10112.0,10274.0,12520.0,12408.0,13118.0,10096,18518
7,Angola,Food,4716.0,4657.0,5124.0,5154.0,5399.0,5549.0,5685.0,5537.0,...,15919.0,16882.0,18243.0,19765.0,21779.0,24465.0,25992.0,27455.0,27968,30121
8,Antigua and Barbuda,Feed,2.0,2.0,2.0,2.0,2.0,0.0,0.0,2.0,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0
9,Antigua and Barbuda,Food,90.0,92.0,103.0,93.0,82.0,73.0,64.0,57.0,...,92.0,113.0,108.0,122.0,115.0,114.0,115.0,118.0,113,119


In [18]:
now['total'] = (now['Y1961'] + now['Y1962'] + now['Y1963'] + now['Y1964'] + now['Y1965'] + now['Y1966'] + 
    now['Y1967'] + now['Y1968'] + now['Y1969'] + now['Y1970'] + now['Y1971'] + now['Y1972'] + now['Y1973'] +
    now['Y1974'] + now['Y1975'] + now['Y1976'] + now['Y1977'] + now['Y1978'] + now['Y1979'] + now['Y1980'] + 
    now['Y1981'] + now['Y1982'] + now['Y1983'] + now['Y1984'] + now['Y1985'] + now['Y1986'] + now['Y1987'] + 
    now['Y1988'] + now['Y1989'] + now['Y1990'] + now['Y1991'] + now['Y1992'] + now['Y1993'] + now['Y1994'] + 
    now['Y1995'] + now['Y1996'] + now['Y1997'] + now['Y1998'] + now['Y1999'] + now['Y2000'] + 
    now['Y2001'] + now['Y2002'] + now['Y2003'] + now['Y2004'] + now['Y2005'] + now['Y2006'] + now['Y2007'] + 
    now['Y2008'] + now['Y2009'] + now['Y2010'] + now['Y2011'] + now['Y2012'] + now['Y2013'])


Classification

In [54]:
Aggr_sum_no_att.head()

,Area,Item,Element,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,Afghanistan,Alcoholic Beverages,Food,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3,3
1,Afghanistan,Animal fats,Food,20.0,20.0,21.0,22.0,24.0,25.0,27.0,...,47.0,46.0,43.0,44.0,46.0,48.0,51.0,52.0,54,69
2,Afghanistan,Apples and products,Food,14.0,14.0,14.0,17.0,19.0,22.0,24.0,...,18.0,18.0,47.0,45.0,49.0,61.0,65.0,56.0,43,68
3,Afghanistan,Bananas,Food,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,1.0,11.0,8.0,12.0,38.0,21.0,28.0,48,82
4,Afghanistan,Barley and products,Feed,76.0,76.0,76.0,76.0,76.0,75.0,71.0,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360


In [55]:
Aggr_sum_no_att1 = pd.get_dummies(Aggr_sum_no_att.Area, drop_first = True)

In [56]:
Aggr_sum_no_att1.head()

,Albania,Algeria,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,...,United Republic of Tanzania,United States of America,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Yemen,Zambia,Zimbabwe
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
Aggr_sum_no_att2 = pd.get_dummies(Aggr_sum_no_att.Item, drop_first = True)

In [58]:
Aggr_sum_no_att2.head()

,Animal fats,Apples and products,"Aquatic Animals, Others",Aquatic Plants,"Aquatic Products, Other",Bananas,Barley and products,Beans,Beer,"Beverages, Alcoholic",...,"Sweeteners, Other",Tea (including mate),Tomatoes and products,Treenuts,Vegetable Oils,Vegetables,"Vegetables, Other",Wheat and products,Wine,Yams
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
frames = [Aggr_sum_no_att, Aggr_sum_no_att1, Aggr_sum_no_att2]

Aggr_sum_no_att_final = pd.concat(frames, sort = False, axis = 1)

In [60]:
Aggr_sum_no_att_final.head()

,Area,Item,Element,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,...,"Sweeteners, Other",Tea (including mate),Tomatoes and products,Treenuts,Vegetable Oils,Vegetables,"Vegetables, Other",Wheat and products,Wine,Yams
0,Afghanistan,Alcoholic Beverages,Food,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Afghanistan,Animal fats,Food,20.0,20.0,21.0,22.0,24.0,25.0,27.0,...,0,0,0,0,0,0,0,0,0,0
2,Afghanistan,Apples and products,Food,14.0,14.0,14.0,17.0,19.0,22.0,24.0,...,0,0,0,0,0,0,0,0,0,0
3,Afghanistan,Bananas,Food,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,Afghanistan,Barley and products,Feed,76.0,76.0,76.0,76.0,76.0,75.0,71.0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
Aggr_sum_no_att.shape

(21018, 56)

In [62]:
Aggr_sum_no_att1.shape

(21018, 173)

In [63]:
Aggr_sum_no_att2.shape

(21018, 114)

In [64]:
Y = (Aggr_sum_no_att['Element'] == 'Food')
Y = Y.astype(int)

In [65]:
X = Aggr_sum_no_att_final.drop(['Element','Area', 'Item'], axis=1)



In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [67]:
#where_are_NaNs = pd.isnull(X)
#X[where_are_NaNs] = 0

In [68]:
#where_are_NaNs = pd.isnull(Y)
#Y[where_are_NaNs] = 0

In [69]:
#Y = np.array(X).reshape(-1,1)

In [70]:
#Y = np.array(Y).reshape(-1,1)

In [71]:
Y.shape

(21018,)

In [72]:
X.shape

(21018, 340)

In [73]:
Y

0        1
1        1
2        1
3        1
4        0
5        1
6        1
7        1
8        1
9        1
10       0
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
20988    1
20989    1
20990    1
20991    1
20992    1
20993    1
20994    0
20995    1
20996    1
20997    1
20998    1
20999    1
21000    1
21001    1
21002    1
21003    1
21004    1
21005    1
21006    1
21007    1
21008    1
21009    1
21010    1
21011    1
21012    1
21013    1
21014    1
21015    1
21016    1
21017    1
Name: Element, Length: 21018, dtype: int32

In [74]:
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [75]:
bnb.score(X_train,y_train)

0.604770665482459

In [76]:
bnb.score(X_test,y_test)

0.6058991436726927

In [77]:
from sklearn.neighbors import KNeighborsClassifier
neighbors = KNeighborsClassifier(n_neighbors=1)

In [78]:
neighbors.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [79]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(neighbors, X_train, y_train, cv=5)

In [80]:
score

array([0.82117945, 0.82334285, 0.82582487, 0.82645939, 0.82170051])

In [81]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(neighbors, X_test, y_test, cv=5)

In [82]:
score

array([0.80323194, 0.79923882, 0.78782112, 0.80685062, 0.81428571])

In [83]:
from sklearn import linear_model

In [84]:
ridgeregr = linear_model.Ridge(alpha=10, fit_intercept=False) 
ridgeregr.fit(X_train, y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=False, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [85]:
ridgeregr.score(X_train,y_train)

0.25873098273270256

In [86]:
ridgeregr.score(X_test,y_test)

0.23392387753742414

In [87]:
# Declare a logistic regression classifier.
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1e9)
# Fit the model.
fit = lr.fit(X_train, y_train)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X)

print('\n Accuracy')
print(pd.crosstab(pred_y_sklearn, Y))

print('\n Percentage accuracy')
print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

c:\users\kristine\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\kristine\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Coefficients
[[ 8.92643565e-04  1.08109232e-03 -1.56846636e-03 -6.19521842e-04
   8.38473580e-04  3.74292235e-04  1.81700420e-03 -1.48690613e-03
  -1.46973645e-03  6.93278042e-04 -4.79927717e-04  4.43457596e-05
  -9.11342516e-04  1.51081981e-03 -1.54495164e-03  2.49765588e-03
   8.00506795e-04 -2.05286207e-03 -1.36309688e-04 -6.89856973e-04
  -3.09164029e-04  1.34644166e-03  1.75832854e-03 -2.99639758e-03
   2.14990861e-04  8.49216831e-04 -1.55475122e-03  1.07598058e-03
  -2.28500370e-04  9.43080145e-05  3.97733755e-04  1.81358807e-04
   1.35295481e-04 -3.41716546e-04 -6.01329928e-04 -8.87457472e-04
   5.47675916e-04  5.41236957e-04  1.28433902e-03  5.81410256e-04
  -1.05116969e-03 -8.27021795e-04  4.97007635e-04 -4.91962571e-04
  -3.47204875e-04  5.96137018e-04  8.24595761e-04 -1.29869702e-03
   1.96213055e-03  2.57589854e-05 -1.15447430e-03  1.90146303e-03
  -1.52349821e-03  1.84007258e-04  1.56958892e-04  1.86745582e-04
   2.13671351e-04  1.70179922e-04  1.61714471e-04  1.68422122e-

## MLP CLASSIFIER

In [88]:
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(2,2))
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [90]:
mlp.score(X_train, y_train)

0.8629068070798706

In [93]:
mlp.score(X_test, y_test)

0.8433872502378686

In [91]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X_train, y_train, cv=5)

array([0.84115409, 0.84205519, 0.81725888, 0.81979695, 0.83978426])

In [92]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X_test, y_test, cv=5)

c:\users\kristine\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
c:\users\kristine\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.83269962, 0.81826832, 0.81826832, 0.81826832, 0.81809524])

In [94]:
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(2,20))
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [97]:
mlp.score(X_train, y_train)


0.8645562392945505

In [96]:
mlp.score(X_test, y_test)

0.8464319695528069

In [98]:
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(20,200))
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 200), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [99]:
mlp.score(X_train, y_train)


0.8881558079045867

In [100]:
mlp.score(X_test, y_test)

0.8588011417697431